## Apartado 1 - Clasificación de texto según el autor
Construid dos modelos de diferentes autores: Josep Carner y Miquel dels Sants Oliver. Para ello, podéis usar sus obras disponibles en el Proyecto Gutenberg. Por ejemplo, de Carner, tomad la traducción de los cuentos de Mark Twain. Luego, clasificad frases en el estilo de cada uno que muestren cómo vuestro modelo las identifica correctamente.

In [4]:
# from my_utils import open_data
from text import *

hostal = open("./CA-text/hostal.txt").read()
wordseq = words(hostal)

P_Abbott = UnigramWordModel(wordseq, 5)

adventures = open("./CA-text/adventures.txt").read()
wordseq = words(adventures)

P_Austen = UnigramWordModel(wordseq, 5)

In [ ]:
from learning import NaiveBayesLearner

dist = {('Abbott', 1): P_Abbott, ('Austen', 1): P_Austen}

nBS = NaiveBayesLearner(dist, simple=True)